## Part 1: Preprocessing

In [33]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

In [34]:
#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [35]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [36]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[["Attrition", "Department"]]

In [37]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',
             'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
             'YearsSinceLastPromotion', 'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
print(X_df.dtypes)

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object


In [38]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)


In [39]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_df["OverTime"].value_counts()

OverTime
No     1054
Yes     416
Name: count, dtype: int64

In [40]:
X_df["OverTime"] = X_df["OverTime"].map({"Yes": 1, "No": 0})
X_train["OverTime"] = X_train["OverTime"].map({"Yes": 1, "No": 0})
X_test["OverTime"] = X_test["OverTime"].map({"Yes": 1, "No": 0})

C:\Users\fsgin\AppData\Local\Temp\ipykernel_16624\1252069082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df["OverTime"] = X_df["OverTime"].map({"Yes": 1, "No": 0})


In [41]:
# Create a StandardScaler
X_scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [42]:
y_df["Department"].value_counts()

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [43]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[["Department"]])
department_columns = department_encoder.get_feature_names_out(['Department'])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded_department = pd.DataFrame(department_encoder.transform(y_train[["Department"]]), columns=department_columns)
y_test_encoded_department = pd.DataFrame(department_encoder.transform(y_test[["Department"]]),  columns=department_columns)

In [44]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[["Attrition"]])
attrition_columns = attrition_encoder.get_feature_names_out(['Attrition'])

# Create two new variables by applying the encoder
# to the training and testing data

y_train_encoded_attrition = pd.DataFrame(attrition_encoder.transform(y_train[["Attrition"]]), columns=attrition_columns)
y_test_encoded_attrition = pd.DataFrame(attrition_encoder.transform(y_test[["Attrition"]]), columns=attrition_columns)


## Create, Compile, and Train the Model

In [45]:
# Find the number of columns in the X training data
input_dim = len(X_columns)

# Create the input layer
input_layer = layers.Input(shape=(input_dim,), name='input_features')

# Create at least two shared layers
shared1 = layers.Dense(64, activation="relu", name="shared1")(input_layer)
shared2 = layers.Dense(128, activation="relu", name='shared2')(shared1)

In [46]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_dense = layers.Dense(32, activation="relu", name='department_hidden')(shared2)

# Create the output layer
department_output = layers.Dense(
    len(y_train_encoded_department.nunique()),
    activation = 'softmax',
    name = 'department_output'
)(department_dense)


In [47]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_dense = layers.Dense(32, activation="relu", name='attrition_hidden')(shared2)

# Create the output layer
attrition_output = layers.Dense(
    len(y_train_encoded_attrition.nunique()),
    activation = 'sigmoid',
    name = 'attrition_output'
)(attrition_dense)


In [48]:
# Create the model
model = Model(
    inputs = input_layer,
    outputs = [attrition_output, department_output],
)

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'department_output' : 'categorical_crossentropy',
        'attrition_output' : "binary_crossentropy",
    }
)

# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared1 (Dense)     │ (None, 64)        │        704 │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared2 (Dense)     │ (None, 128)       │      8,320 │ shared1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_hidde… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
# Train the model
model.fit(
    X_train_scaled,
    {
        'attrition_output' : y_train_encoded_attrition,
        'department_output' : y_train_encoded_department
    },
    epochs=100,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - attrition_output_loss: 0.6377 - department_output_loss: 0.9733 - loss: 1.6115 - val_attrition_output_loss: 0.5017 - val_department_output_loss: 0.8553 - val_loss: 1.3590
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_loss: 0.4620 - department_output_loss: 0.7949 - loss: 1.2568 - val_attrition_output_loss: 0.4733 - val_department_output_loss: 0.8373 - val_loss: 1.3122
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_loss: 0.3983 - department_output_loss: 0.7178 - loss: 1.1163 - val_attrition_output_loss: 0.4584 - val_department_output_loss: 0.8425 - val_loss: 1.3022
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - attrition_output_loss: 0.3588 - department_output_loss: 0.7479 - loss: 1.1065 - val_attrition_output_loss: 0.4550 - val_department_output_loss: 0.8516 - val_loss: 1.3078
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_loss: 0.3427 - department_output_loss

In [50]:
# Evaluate the model with the testing data
results = model.evaluate(np.array(X_test_scaled), {
    'attrition_output': y_test_encoded_attrition,
    'department_output': y_test_encoded_department
})
print(results)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - attrition_output_loss: 1.1963 - department_output_loss: 3.2641 - loss: 4.4760
[4.52260684967041, 1.2544933557510376, 3.166081666946411]


In [51]:
# Print the accuracy for both department and attrition
from sklearn.metrics import accuracy_score
attrition_predictions, department_predictions = model.predict(X_test_scaled)

attrition_predictions_adj = np.argmax(attrition_predictions, axis=1)
department_predictions_adj = np.argmax(department_predictions, axis=1)

y_test_attrition_labels = np.argmax(y_test_encoded_attrition, axis=1) if len(y_test_encoded_attrition.shape) > 1 else y_test_encoded_attrition
y_test_department_labels = np.argmax(y_test_encoded_department, axis=1) if len(y_test_encoded_department.shape) > 1 else y_test_encoded_department

attrition_accuracy = accuracy_score(y_test_attrition_labels, attrition_predictions_adj)
department_accuracy = accuracy_score(y_test_department_labels, department_predictions_adj)

print(f"Attrition predictions accuracy: {attrition_accuracy}")
print(f"Department predictions accuracy: {department_accuracy}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Attrition predictions accuracy: 0.7907608695652174
Department predictions accuracy: 0.5407608695652174


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

In [52]:
y_df.value_counts()

Attrition  Department            
No         Research & Development    828
           Sales                     354
Yes        Research & Development    133
           Sales                      92
No         Human Resources            51
Yes        Human Resources            12
Name: count, dtype: int64

YOUR ANSWERS HERE

1. Accuracy is not the best metric for this data. The data is very imbalanced, and I 
   did not adjust or account for it in the model.
2. For attrition, I used sigmoid since it is a binary classification.
   For department, I used softmax because it is multiclass classification, and I applied
   a reasonable assumption that you only belong to one department.
3. I could spend time testing out different hidden layer activation funcations, 
   fiddle with the neurons in each layer, and try out various other metrics. In addition, 
   I could try it with various numbers of epochs and differing batch sizes. Finally, I 
   could see about changing which X features I want to run this on, as well as balence the y's.